In [122]:
#include json and os librareries
import json
import time
import pandas as pd

In [162]:
####Globals:
JSON_DIR="ExtractTestsResults/JsonTestsFolders/cpuUtil/cpu_util-"


############################################################################
#                                                                          #
#                    #################################                     #
#                    #      Helper functions         #                     #
#                    #################################                     #
#                                                                          #
############################################################################


def add_to_os_daemon_list(os_daemon_list:dict, row):
    """
    insert a row to the os daemon list\n
    params:
        -os_daemon_list(list)-a list that contain all the oc daemon components of a iteration of ocp version with specific build.\n
        -row-pandas row
    returns: the updated os daemon dict.
    """
    group_name = row["Component"].split('"')[1]
    max_cpu = row["CPU"]
    os_daemon_list.append({"group_name":group_name, "avg_cpu":max_cpu})

    return os_daemon_list


def add_to_infra_pods_list(infra_pods_list:list, row):
    """
    insert a row to the infra pods list\n
    params:
        -infra_pods_list(list)-a list that contain all the inftra pods components of a iteration of ocp version with specific build.\n
        -row-pandas row
    returns: the updated infra pods dict.
    """
    namespace = row["Component"].split('=')[1].split(",")[0].replace("\"", "")
    pod = row["Component"].split('=')[2].split("}")[0].replace("\"", "")
    max_cpu = row["CPU"]
    infra_pods_list.append({"namespace":namespace, "pod":pod,"avg_cpu":max_cpu})

    return infra_pods_list


def iter_grouped_components(iter_components_result_df):
    """
    extract all the fields from a grouped components(rows are grouped by the same ocp verison, ocp build and iteration),\n
    params:
        -iter_components_result_df(df)-grouped dataframe
    returns: oc daemon and infra pods lists, duration, avg_cpu_total and node name, iteration, kernel and sideloaded\n.
    """
    infra_pods_list = []
    os_daemon_list = []
    #get global params:
    steady_total_avg =  iter_components_result_df["Steady Workloads - Avg"].unique()
    avg_cpu_total = steady_total_avg[0] if steady_total_avg[0] != "" else steady_total_avg[1]
    duration = iter_components_result_df["Duration"].unique()[0]
    iteration = int(iter_components_result_df["Iteration"].unique()[0])
    node_name = iter_components_result_df["Cluster"].unique()[0]
    kernel = iter_components_result_df["Kernel"].unique()[0] if "Kernel" in(iter_components_result_df.columns) else ""
    sideloaded = iter_components_result_df["Sideloaded"].get(key=0, default=None) if "Sideloaded" in(iter_components_result_df.columns) else ""
    if len(sideloaded) > 0:
        sideloaded = "true" if sideloaded == "Yes" else "false"
    #update components lists
    for _, row in iter_components_result_df.iterrows():
        
        #update components lists
        if "namespace=" in row["Component"]:
            infra_pods_list = add_to_infra_pods_list(infra_pods_list=infra_pods_list,row=row)
        else: 
            os_daemon_list = add_to_os_daemon_list(os_daemon_list=os_daemon_list, row=row)
    
    return infra_pods_list, os_daemon_list, avg_cpu_total, duration, node_name, kernel, sideloaded, iteration

def extract_types_of_scenario(results_df):
    """
    extract all types metrics of specific scenario.\n
    params:
        -results_df(df)-grouped dataframe by the same scenario value
    returns: list of types metrics
    """
    types_list = []
    total = results_df["Max CPU"].unique()
    total_max_cpu = total[0] if total[0] != "" else total[1]
    for _, row in results_df.iterrows():
        types_list.append({"type_name":row["Type"], "max_cpu":row["CPU"]})
    
    #add total metrics
    types_list.append({"type_name":"total", "max_cpu":total_max_cpu})

    return types_list

def create_json(results_dict:dict, JSON_DIR:str):
    #create json object
    #add uniuqe id to file name (using unix time)
    timestamp = str(time.time()).replace('.', '')
    # open a file and write the JSON data to it
    with open(JSON_DIR+timestamp+".json", "w") as json_file:
        json.dump(results_dict, json_file)


def add_steadyworkload_metrics(scenario_dict:dict, os_daemon_list:list, infra_pods_list:list, avg_cpu_total:float, duration:str):
    scenario_dict["components_os_daemon"] = os_daemon_list
    scenario_dict["components_infra_pods"] = infra_pods_list
    scenario_dict["avg_cpu_total"] = avg_cpu_total
    scenario_dict["duration"] = duration
    return scenario_dict


In [169]:
cpu_components_df = pd.read_csv("RAN QE CPU Utilization - component.csv").fillna("")
cpu_scenarios_df = pd.read_csv("RAN QE CPU Utilization - other.csv").fillna("")
cpu_components_df["OCP Version"] = cpu_components_df["OCP Version"].astype("str")
cpu_scenarios_df["OCP Version"] = cpu_scenarios_df["OCP Version"].astype("str")

In [173]:
grouped_components = cpu_components_df.groupby(["Version", "OCP Version", "Iteration"])
grouped_scenarios = cpu_scenarios_df.groupby(["Version", "OCP Version", "Iteration"])

In [174]:
for version_iter_key, iter_components_result_df in grouped_components:
    print("creating json file for {}".format(version_iter_key))
    ocp_version = str(version_iter_key[0])
    ocp_build = str(version_iter_key[1])
    #extract global params and components values
    infra_pods_list, os_daemon_list, avg_cpu_total, duration, node_name, kernel, sideloaded, iteration = iter_grouped_components(iter_components_result_df)
    #extract scenarios metrics
    iter_scenarios_result_df = grouped_scenarios.get_group(version_iter_key) if version_iter_key in grouped_scenarios.groups else None
    scenarios_list = []
    if iter_scenarios_result_df is not None:
        for scenario, results in iter_scenarios_result_df.groupby("Scenario"):
            scenario_dict = {}
            scenario_dict["scenario_name"] = scenario
            scenario_dict["types"] = extract_types_of_scenario(results_df=results)
            if scenario == "steadyworkload":
                scenario_dict = add_steadyworkload_metrics(scenario_dict=scenario_dict, os_daemon_list=os_daemon_list, 
                                            infra_pods_list=infra_pods_list, avg_cpu_total=avg_cpu_total, 
                                            duration=duration)
            scenarios_list.append(scenario_dict)
    else:
        scenario_dict = {}
        scenario_dict["scenario_name"] = "steadyworkload"
        scenario_dict = add_steadyworkload_metrics(scenario_dict=scenario_dict, os_daemon_list=os_daemon_list, 
                                            infra_pods_list=infra_pods_list, avg_cpu_total=avg_cpu_total, 
                                            duration=duration)
        scenarios_list.append(scenario_dict)
        

    results_dict = {"ocp_version":ocp_version, "ocp_build":ocp_build, "test_type":"cpu_util",\
                    "iteration":iteration, "node_name":node_name, "kernel":kernel, "sideloaded":sideloaded, "scenarios":scenarios_list}
    create_json(results_dict=results_dict, JSON_DIR=JSON_DIR)
    

creating json file for (4.11, '4.11.22', 1)
